In [1]:
# import sympy and numpy 
from sympy import *
from sympy.physics.quantum import *
import numpy as np
from IPython.display import Math
from functools import reduce

# reference in current ISA paper, gate section

Let $A, B, C \in \text{GL}_{\text{n}}(\mathbb{C})$ . Recall that 

$$\Large e^{A + B} = \lim_{m \to \infty}\left( e^{\frac{A}{m}}e^{\frac{B}{m}}\right)^m $$

which means 

$$ \Large e^{A + B} \approx \left( e^{\frac{A}{m}}e^{\frac{B}{m}}\right)^m $$

for sufficiently large $ m $. 

If $\left[ A, B\right] = 0 $ then 

$$ \Large e^{A + B + C} = \lim_{m \to \infty}\left( e^{\frac{A}{m}}e^{\frac{B}{m}}e^{\frac{C}{m}}\right)^m $$

then 

$$ \Large e^{A + B + C} \approx \left( e^{\frac{A}{m}}e^{\frac{B}{m}}e^{\frac{C}{m}}\right)^m $$

for sufficiently large $m$.  

# Functions for Trotterization and series expansion

In [2]:
# define symbolic matrices, scalars, and constants 
A = MatrixSymbol('A', 2, 2)
B = MatrixSymbol('B', 2, 2)
C = MatrixSymbol('C', 2, 2)
x, θ, λ, n, t, approx, equal = symbols('x, θ, λ, n, t, ≈, =')
i = sqrt(-1)
π = pi

# define Pauli matrices
σ_x = Matrix([[0, 1], [1, 0]])
σ_y = Matrix([[0, -I], [I, 0]])
σ_z = Matrix([[1, 0], [0, -1]])

In [3]:
# trotterization with matrices A, B 
def trotterize(A, B, variable, n, expand = True):
    """
    function to Trotterize matrix exponential up to order n, 
    where the variable we tune in the exponent is given by variable
    """
    
    # display the matrix expnonential we approximate
    matrix_exponential = exp(-i*variable*(A + B))
    latex_expression1 = latex(matrix_exponential) + latex(approx)
    
    # compute trotter formula
    trotter_formula = (exp((-i*variable*A)/n)*exp((-i*variable*B)/n))**n
    
    if expand:
        # return Trotterization approximation 
        latex_expression2 = latex(trotter_formula) + latex(equal)
        display(Math(latex_expression1 + latex_expression2))
        return trotter_formula.expand()
    else: 
        display(Math(latex_expression1))
        return trotter_formula
display(trotterize(A, B, θ, 5, expand = True))

<IPython.core.display.Math object>

exp((-I*θ/5)*A)*exp((-I*θ/5)*B)*exp((-I*θ/5)*A)*exp((-I*θ/5)*B)*exp((-I*θ/5)*A)*exp((-I*θ/5)*B)*exp((-I*θ/5)*A)*exp((-I*θ/5)*B)*exp((-I*θ/5)*A)*exp((-I*θ/5)*B)

In [4]:
# trotterization with matrices A, B, C where [A, C] = 0 
def trotterize_neighbor(A, B, C, n, expand = True):
    """
    function to Trotterize matrix exponential up to order n 
    whose Hamiltonian involves neighboring sites; 
    
    this function assumes [A, B] = 0
    """
    # display the matrix expnonential we approximate
    matrix_exponential = exp(-i*θ*(A + B + C))
    latex_expression1 = latex(matrix_exponential) + latex(approx)
    
    # compute trotter formula
    trotter_formula = (exp((-i*θ*A)/n)*exp((-i*θ*B)/n)*exp((-i*θ*C)/n))**n
    
    if expand:
        # return Trotterization approximation 
        latex_expression2 = latex(trotter_formula) + latex(equal)
        display(Math(latex_expression1 + latex_expression2))
        return trotter_formula.expand()
    else: 
        display(Math(latex_expression1))
        return trotter_formula
display(trotterize_neighbor(A, B, C, 4, expand = True))    

<IPython.core.display.Math object>

exp((-I*θ/4)*A)*exp((-I*θ/4)*B)*exp((-I*θ/4)*C)*exp((-I*θ/4)*A)*exp((-I*θ/4)*B)*exp((-I*θ/4)*C)*exp((-I*θ/4)*A)*exp((-I*θ/4)*B)*exp((-I*θ/4)*C)*exp((-I*θ/4)*A)*exp((-I*θ/4)*B)*exp((-I*θ/4)*C)

In [5]:
# function to taylor expand any matrix
M = Matrix([[cos(x), -sin(x)], [sin(x), cos(x)]])

def expand_exponential(A, x, x_0, order):
    """
    Expands matrix exponential in variable x, about x_0, and to order 'order'

    Args:
        A: matrix we will expand
        x: variable in which we will expand A
        x_0: variable about which we will expand A
        order: the order to which we will truncate the expansion

    Returns:
        returns expanded matrix exponential
    """
    
    # collect rows and columns of matrix A
    rows, columns = A.shape
    
    # create series_matrix with only NaN entries, which we will later fill 
    series_matrix = Matrix(rows, columns, lambda i, j: nan)
    
    # expand matrix exponential in x, about x_0, and up to order 'order'
    for (row_index, column_index), element in np.ndenumerate(A):
        expanded_element = element.series(x, x_0, order + 1)
        series_matrix[row_index, column_index] = expanded_element

    # return series_matrix only if all NaNs have been replaced
    if not series_matrix.has(nan):
        # display matrix
        display(Math(latex(A) + latex(equal)))
        return series_matrix
    else:
        print('There are NaNs that must be replaced')
expand_exponential(M, x, 0, 10)

<IPython.core.display.Math object>

Matrix([
[1 - x**2/2 + x**4/24 - x**6/720 + x**8/40320 - x**10/3628800 + O(x**11),             -x + x**3/6 - x**5/120 + x**7/5040 - x**9/362880 + O(x**11)],
[             x - x**3/6 + x**5/120 - x**7/5040 + x**9/362880 + O(x**11), 1 - x**2/2 + x**4/24 - x**6/720 + x**8/40320 - x**10/3628800 + O(x**11)]])

# Simulation of $ U_N $

For a chain of N spins, let us define $R_{XX}(\theta), R_{YY}(\theta), \text{ and } R_{ZZ}(\theta)$ acting on the jth and (j+1)th sites as 

$$ \Large R_{XX}^{ \left(j \right)}\left(\theta \right) = \mathbb{1}^{\otimes j -1} \otimes e^{\frac{-i \theta}{2} X_j \otimes X_{j+1}} \otimes \mathbb{1}^{\otimes N - j -1} $$
$$ \Large R_{YY}^{ \left(j \right)}\left(\theta \right) = \mathbb{1}^{\otimes j -1} \otimes e^{\frac{-i \theta}{2} Y_j \otimes Y_{j+1}} \otimes \mathbb{1}^{\otimes N - j -1} $$
$$ \Large R_{ZZ}^{ \left(j \right)}\left(\lambda \theta \right) = \mathbb{1}^{\otimes j -1} \otimes e^{\frac{-i \lambda \theta}{2} Z_j \otimes Z_{j+1}} \otimes \mathbb{1}^{\otimes N - j -1} $$

so that 

$$ \Large R_{XYZ}^{ \left(j \right)}\left(\theta, \lambda \right) = R_{XX}^{ \left(j \right)}\left(\theta \right) R_{YY}^{ \left(j \right)}\left(\theta \right) R_{ZZ}^{ \left(j \right)}\left(\lambda \theta \right) $$

We now define 

$$ \Large R_B(\theta, \lambda) =  \prod_{j=1}^{\lceil \frac{N - 1}{2} \rceil} R_{XYZ}^{ \left(2j - 1 \right)}\left(\theta, \lambda \right) $$
$$ \Large R_R(\theta, \lambda) =  \prod_{j=1}^{\lfloor \frac{N - 1}{2} \rfloor} R_{XYZ}^{ \left(2j \right)}\left(\theta, \lambda \right) $$

where $ R_B(\theta)$ acts on all sites connected by red bonds and $ R_R(\theta) $ acts on all sites connected by red bonds. Since $ R_B(\theta) $ and $ R_R(\theta) $ act simultaneously on all bonds in the lattice, we approximate the unitary evolution of the system by 

$$ \Large U \approx \left( R_B(\theta, \lambda)  R_R(\theta, \lambda) \right)^m $$

In [6]:
dv_Rxx, dv_Ryy, dv_Rzz, dv_Rxyz = symbols('R_{XX}(θ), R_{YY}(θ), R_{ZZ}(θ), R_{XYZ}(θ)')

# define RXX, RYY and RZZ gates with a matrix exponential; this agrees with Qiskit
Rxx_exp = exp(-I*θ/2*TensorProduct(σ_x, σ_x)).simplify()
Ryy_exp = exp(-I*θ/2*TensorProduct(σ_y, σ_y)).simplify()
Rzz_exp = exp(-I*θ/2*TensorProduct(σ_z, σ_z))

# define RXX, RYY, and RZZ gates manually
Rxx = Matrix([[cos(θ/2), 0, 0, -I*sin(θ/2)], 
               [0, cos(θ/2), -I*sin(θ/2), 0], 
               [0, -I*sin(θ/2), cos(θ/2), 0], 
               [-I*sin(θ/2), 0, 0, cos(θ/2)]])

Ryy = Matrix([[cos(θ/2), 0, 0, I*sin(θ/2)], 
               [0, cos(θ/2), -I*sin(θ/2), 0], 
               [0, -I*sin(θ/2), cos(θ/2), 0], 
               [I*sin(θ/2), 0, 0, cos(θ/2)]])

Rzz = Matrix([[exp(- I*θ/2), 0, 0, 0], 
               [0, exp(I*θ/2), 0, 0], 
               [0, 0, exp(I*θ/2), 0], 
               [0, 0, 0, exp(-I*θ/2)]])

# define Rxyz gate
Rxyz = simplify(Rxx*Ryy*Rzz)

# display RXX, RYY and RZZ gates
if Rxx == Rxx_exp and Ryy == Ryy_exp and Rzz == Rzz_exp:
    Rxx_equation = Eq(dv_Rxx, Rxx, evaluate = False)
    Ryy_equation = Eq(dv_Ryy, Ryy, evaluate = False)
    Rzz_equation = Eq(dv_Rzz, Rzz, evaluate = False)
    Rxyz_equation = Eq(dv_Rxyz, Rxyz, evaluate = False)
    display(Math(latex(Rxx_equation) + latex(Ryy_equation)))
    display(Math(latex(Rzz_equation)))
    display(Math(latex(Rxyz_equation)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [ ]:
exp(-I*θ/2*(TensorProduct(σ_x, σ_x) + TensorProduct(σ_z, σ_z)))

In [7]:
# function to generate H_blue and H_red for a lattice of N_sites
def generate_bue_red_operators(N, explicit_tensor_product):
    
    if explicit_tensor_product == True: 
        
        # define RXX, RYY, and RZZ gates manually so that H_red 
        # and H_blue will have dimension 4^N
        RXX = Matrix([[cos(θ/2), 0, 0, -I*sin(θ/2)], 
                       [0, cos(θ/2), -I*sin(θ/2), 0], 
                       [0, -I*sin(θ/2), cos(θ/2), 0], 
                       [-I*sin(θ/2), 0, 0, cos(θ/2)]])

        RYY = Matrix([[cos(θ/2), 0, 0, I*sin(θ/2)], 
                       [0, cos(θ/2), -I*sin(θ/2), 0], 
                       [0, -I*sin(θ/2), cos(θ/2), 0], 
                       [I*sin(θ/2), 0, 0, cos(θ/2)]])

        RZZ = Matrix([[exp(-I*λ*θ/2), 0, 0, 0], 
                       [0, exp(I*λ*θ/2), 0, 0], 
                       [0, 0, exp(I*λ*θ/2), 0], 
                       [0, 0, 0, exp(-I*λ*θ/2)]])
        
    elif explicit_tensor_product == False:
        
        # define RXX, RYY and RZZ gates with a matrix exponential
        # so that H_blue and H_red will have \otimes between Hilbert spaces
        RXX = exp(-I*θ/2*TensorProduct(σ_x, σ_x)).simplify()
        RYY = exp(-I*θ/2*TensorProduct(σ_y, σ_y)).simplify()
        RZZ = exp(-I*λ*θ/2*TensorProduct(σ_z, σ_z))


    # create empty lists for RXX, RYY, and RZZ operators 
    RXX_list, RYY_list, RZZ_list = [], [], []
    
    # display(TensorProduct(RXX, RZZ))

    for i in range(N):

        # define iterator that begins at 1 and ends at N
        j = i + 1

        if N == 1: 
            print("Please enter valid N greater than 2")
        elif j < N: 

            # creates identity matrices that will precede and follow 
            # the location of the RXX, RYY, or RZZ gate
            pre_identities = [eye(2) for site in range(j - 1)]
            post_identities = [eye(2) for site in range(N-j-1)]

            # create RXX_j depending on value of j 
            if j == 1: 
                RXX_j = [RXX] + post_identities
                RYY_j = [RYY] + post_identities
                RZZ_j = [RZZ] + post_identities
            elif j == (N - 1): 
                RXX_j = pre_identities + [RXX] 
                RYY_j = pre_identities + [RYY]
                RZZ_j = pre_identities + [RZZ]
            else: 
                RXX_j = pre_identities + [RXX] + post_identities
                RYY_j = pre_identities + [RYY] + post_identities
                RZZ_j = pre_identities + [RZZ] + post_identities

            # append list of matrices to RXX_list, RYY_list, and RZZ_list
            RXX_list.append(RXX_j)
            RYY_list.append(RYY_j)
            RZZ_list.append(RZZ_j)
    
    # compute the tensor product of each element in RXX_tensored_list, RYY_tensored_list, and RZZ_tensored_list; 
    # note for example that RZZ_tensored_list[j] imparts an RZZ gate on the jth and jth + 1
    # spin in a spin chain of N sites
    RXX_tensored_list = [TensorProduct(*element) for element in RXX_list]
    RYY_tensored_list = [TensorProduct(*element) for element in RYY_list]
    RZZ_tensored_list = [TensorProduct(*element) for element in RZZ_list]

    # the "zip" iterator that produces tuples containing elements from the input iterables; 
    # here, we multiply elementwise the RXX, RYY and RZZ gates on the jth and jth + 1 spin so that 
    # RXYZ_tensored_list[j] imparts an RZZ, RYY, and then RXX gate upon the jth and jth + 1 spins
    RXYZ_list = [simplify(xx*yy*zz) for xx, yy, zz in zip(RXX_tensored_list, RYY_tensored_list, RZZ_tensored_list)]

    # multiply all RXYZ operators acting on odd lattice sites; this is an operator that applies
    # an RZZ, RYY, then RXX rotation to all "blue bonds" in a 1D spin chain  
    if N == 2: 
        RXYZ_odd_sites = Mul(*RXYZ_list)
        
        # return only Hamiltonian that for one bond 
        return RXYZ_odd_sites, None
    else: 
    
        RXYZ_odd_sites = reduce(lambda x, y: x*y, [matrix for index, matrix in enumerate(RXYZ_list) if index % 2 == 0])

        # multiply all RXYZ operators acting on even lattice sites; this is an operator that applies
        # an RZZ, RYY, then RXX rotation to all "red bonds" in a 1D spin chain 
        RXYZ_even_sites = reduce(lambda x, y: x*y, [matrix for index, matrix in enumerate(RXYZ_list) if index % 2 != 0])

        return RXYZ_odd_sites, RXYZ_even_sites
    
# create function that trotterizes two matrices and returns 
trotter = lambda A, B, m: simplify((A*B)**m)

N = 3
R_B, R_R = generate_bue_red_operators(N, explicit_tensor_product = True)

In [ ]:
trotter(R_B, R_R, 3)

In [860]:
expand_exponential(trot, θ, 0, 3)

<IPython.core.display.Math object>

Matrix([
[1 - 3*I*θ*λ - 9*θ**2*λ**2/2 + 9*I*θ**3*λ**3/2 + O(θ**4),                                                      0,                                                                          0,                                                     0,                                                     0,                                                                          0,                                                      0,                                                       0],
[                                                      0,                    1 - 9*θ**2/2 - 4*I*θ**3*λ + O(θ**4),                      -3*I*θ + 3*θ**2*λ + θ**3*(5*I*λ**2/2 + 7*I) + O(θ**4),                                                     0,                          -3*θ**2 - I*θ**3*λ + O(θ**4),                                                                          0,                                                      0,                                                       0],
[          

# $e^{-i\hat{H}t} = e^{-i\left( \hat{H}_0 + \hat{H}_1 \right)t} \approx \left( e^{-\frac{i\hat{H}_0 t}{m}} e^{-\frac{i\hat{H}_1 t}{m}} \right)^m $

In [9]:
# define symbolic scalars and "dummy variables" for matrices 
dv_U, dv_U_0, dv_U_1, dv_H, dv_H_0, dv_H_1, dv_II, dv_XX, dv_YY, dv_ZZ, t, λ = symbols(
    """ \hat{U}, \hat{U}_0, \hat{U}_1, \hat{H}, \hat{H}_0, \hat{H}_1, \hat{I}_1\hat{I}_2, 
    \hat{X}_{1}\hat{X}_{2}, \hat{Y}_{1}\hat{Y}_{2}, \hat{Z}_{1}\hat{Z}_{2}, t, λ """)

# define symbolic matrices 
X1 = MatrixSymbol('\hat{X}_1', 4, 4)
X2 = MatrixSymbol('\hat{X}_2', 4, 4)
Y1 = MatrixSymbol('\hat{Y}_1', 4, 4)
Y2 = MatrixSymbol('\hat{Y}_2', 4, 4)
Z1 = MatrixSymbol('\hat{Z}_1', 4, 4)
Z2 = MatrixSymbol('\hat{Z}_2', 4, 4)

# calculate tensor product of II, XX, YY, and ZZ
tensor_II = TensorProduct(eye(2), eye(2))
tensor_XX = TensorProduct(σ_x, σ_x)
tensor_YY = TensorProduct(σ_y, σ_y)
tensor_ZZ = TensorProduct(σ_z, σ_z)

# display tensor products in matrix form 
II_equation = Eq(dv_II, tensor_II, evaluate = False)
XX_equation = Eq(dv_XX, tensor_XX, evaluate = False)
YY_equation = Eq(dv_YY, tensor_YY, evaluate = False) 
ZZ_equation = Eq(dv_ZZ, tensor_ZZ, evaluate = False)
display(Math(latex(II_equation) + latex(XX_equation) + latex(YY_equation) + latex(ZZ_equation)))

<IPython.core.display.Math object>

In [14]:
H = tensor_XX + tensor_YY 
exp(-I*θ/2 *H)

Matrix([
[1,                         0,                         0, 0],
[0,  exp(I*θ)/2 + exp(-I*θ)/2, -exp(I*θ)/2 + exp(-I*θ)/2, 0],
[0, -exp(I*θ)/2 + exp(-I*θ)/2,  exp(I*θ)/2 + exp(-I*θ)/2, 0],
[0,                         0,                         0, 1]])

In [ ]:
# display Hamiltonians 
H_0 = X1*X2 + Y1*Y2
H_1 = λ*Z1*Z2
H = H_0 + H_1
display(Eq(dv_H, dv_H_0 + dv_H_1))
display(Eq(dv_H_0, H_0, evaluate = False))
display(Eq(dv_H_1, H_1, evaluate = False))

# display matrix exponential to Trotterize
U = exp(-i * t * (H_0 + H_1))
display(Math(latex(dv_U) + latex(equal) + latex(U)))

Since $ \left[\hat{X}_1 \hat{X}_2, \hat{Y}_1 \hat{Y}_2 \right] = 0 $ then

In [ ]:
U_trotter = trotterize_neighbor(X1*X2, Y1*Y2, λ*Z1*Z2, 3, expand = True)
display(U_trotter)

In [ ]:
U_trotter = trotterize_neighbor(tensor_XX, tensor_YY, λ*tensor_ZZ,  3, expand = True)
display(U_trotter)

In [ ]:
n = 1
r1 = exp(-i*θ*(tensor_XX + tensor_YY + λ*tensor_ZZ))
r2 = ((exp(-i*θ/n*tensor_XX)*exp(-i*θ/n*tensor_YY)*exp(-λ*i*θ/n*tensor_ZZ))**n).expand()
display(Eq(U, r1, evaluate = False))
display()

In [ ]:
mat2 = trotterize(tensor_XX + tensor_YY, λ*tensor_ZZ, 4, expand = True)

In [ ]:
expand_exponential(U_trotter, θ, 0, 10)

In [ ]:
expand_exponential(mat2, θ, 0, 10)

# Error Analysis

In [ ]:
# define symbolic matrices
H = MatrixSymbol('H', 2, 2)
H_0 = MatrixSymbol('H_0', 2, 2)
H_1 = MatrixSymbol('H_1', 2, 2)
U = MatrixSymbol('U', 2, 2)
U_0 = MatrixSymbol('U_0', 2, 2)
U_1 = MatrixSymbol('U_1', 2, 2)
x = symbols('x')

In [25]:
A = Matrix([[cos(x), -sin(x)], [sin(x), cos(x)]])
A

Matrix([
[cos(x), -sin(x)],
[sin(x),  cos(x)]])

In [26]:
def expand_exponential(A, x, x_0, order):
    """
    Expands matrix exponential in variable x, about x_0, and to order 'order'

    Args:
        A: matrix we will expand
        x: variable in which we will expand A
        x_0: variable about which we will expand A
        order: the order to which we will truncate the expansion

    Returns:
        returns expanded matrix exponential
    """
    # collect rows and columns of matrix A
    rows, columns = A.shape
    
    # create series_matrix with only NaN entries, which we will later fill 
    series_matrix = Matrix(rows, columns, lambda i, j: nan)
    
    # expand matrix exponential in x, about x_0, and up to order 'order'
    for (row_index, column_index), element in np.ndenumerate(A):
        expanded_element = element.series(x, x_0, order)
        series_matrix[row_index, column_index] = expanded_element

    # return series_matrix only if all NaNs have been replaced
    if not series_matrix.has(nan):
        return series_matrix
    else:
        print('There are NaNs that must be replaced')

In [27]:
expand_exponential(A, x, 0, 7)

Matrix([
[1 - x**2/2 + x**4/24 - x**6/720 + O(x**7),          -x + x**3/6 - x**5/120 + O(x**7)],
[          x - x**3/6 + x**5/120 + O(x**7), 1 - x**2/2 + x**4/24 - x**6/720 + O(x**7)]])